In [1]:
import scrape

In [2]:
d = scrape.scrape()

---->     Extracting page:  https://www.jobspider.com/job/view-resume-83965.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83943.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83929.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83923.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83916.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83898.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83896.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83885.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83875.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83874.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-

In [4]:
d.keys().__len__()

26

In [5]:
d['SpiderID'].__len__()

1500

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame.from_dict(d, orient='index').T

In [8]:
df.to_csv('resume_data_improved.csv')

In [ ]:
d.keys()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Role                      1500 non-null   object
 1   Desired Industry          1500 non-null   object
 2   SpiderID                  1500 non-null   object
 3   Desired Job Location      1500 non-null   object
 4   Date Posted               1500 non-null   object
 5   Type of Position          1500 non-null   object
 6   Availability Date         876 non-null    object
 7   Desired Wage:             0 non-null      object
 8   U.S. Work Authorization   1386 non-null   object
 9   Job Level                 1500 non-null   object
 10  Willing to Travel:        0 non-null      object
 11  Highest Degree Attained   1382 non-null   object
 12  Willing to Relocate       1387 non-null   object
 13  Objective                 1291 non-null   object
 14  Experience              

In [ ]:
df.nunique()

In [10]:
df.shape

(1500, 26)

In [ ]:
df.drop

In [ ]:
df1 = pd.read_csv('resume_data.csv')

In [11]:
df1['Industry'].nunique()

NameError: name 'df1' is not defined

In [12]:
df.isnull().sum() / df.shape[0]

Role                        0.000000
Desired Industry            0.000000
SpiderID                    0.000000
Desired Job Location        0.000000
Date Posted                 0.000000
Type of Position            0.000000
Availability Date           0.416000
Desired Wage:               1.000000
U.S. Work Authorization     0.076000
Job Level                   0.000000
Willing to Travel:          1.000000
Highest Degree Attained     0.078667
Willing to Relocate         0.075333
Objective                   0.139333
Experience                  0.061333
Education                   0.102667
Affiliations                0.583333
Skills                      0.128000
Additional Information      0.391333
Desired Wage                0.301333
Willing to Travel           0.302667
Availability Date:          1.000000
U.S. Work Authorization:    1.000000
Highest Degree Attained:    1.000000
Willing to Relocate:        1.000000
Reference                   0.674000
dtype: float64